In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
from scipy.stats import norm
import seaborn as sns
import matplotlib.pyplot as plt
import os
import openai

import pandas as pd
import os
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
import bokeh
import os
import openai
from openai import OpenAI
import backoff
import numpy as np
from sklearn.cluster import KMeans
import backoff
import requests 

In [2]:
# Load embeddings file

# input the path to your saved embeddings
path = '/Users/ellenbowen/Desktop/Michigan/Siwo_lab/'
file = str(path) + 'phecode_gpt_embedding.pkl'

embeddings_df = pd.read_pickle(file)

In [3]:
embeddings_df

,phecode,phenotype,phecode_exclude_range,sex,rollup,leaf,category_number,category,embeddings
0,8.00,Intestinal infection,001-009.99,NaN,1,0,1,infectious diseases,"[0.0068802302703261375, 0.0005461262771859765,..."
1,8.50,Bacterial enteritis,001-009.99,Both,1,0,1,infectious diseases,"[-0.010700925253331661, 1.566567698318977e-05,..."
2,8.51,Intestinal e.coli,001-009.99,Both,1,1,1,infectious diseases,"[0.01900758408010006, -0.0076250554993748665, ..."
3,8.52,Intestinal infection due to C. difficile,001-009.99,Both,1,1,1,infectious diseases,"[0.013732682913541794, -0.013513063080608845, ..."
4,8.60,Viral Enteritis,001-009.99,Both,1,1,1,infectious diseases,"[-0.01282576099038124, -0.0019294519443064928,..."
...,...,...,...,...,...,...,...,...,...
1861,994.00,Sepsis and SIRS,994-994.99,NaN,1,0,18,injuries & poisonings,"[-0.009411283768713474, -0.011322950012981892,..."
1862,994.10,Systemic inflammatory response syndrome (SIRS),994-994.99,Both,1,1,18,injuries & poisonings,"[-0.036918796598911285, -0.012674717232584953,..."
1863,994.20,Sepsis,994-994.99,Both,1,0,18,injuries & poisonings,"[0.004166525788605213, -0.005852119065821171, ..."
1864,994.21,Septic shock,994-994.99,Both,1,1,18,injuries & poisonings,"[-0.012952380813658237, 0.0007000957266427577,..."


In [4]:
#define API from global variable

openai.api_key = os.environ.get("OPENAI_API_KEY")

# try and catch to call OpenAI API

@backoff.on_exception(backoff.expo, requests.exceptions.RequestException, max_tries=8)
def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    try:
        return openai.embeddings.create(input=[text], model=model).data[0].embedding
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        raise  # Re-raise the exception to trigger backoff




In [5]:
mendelian_dx = ['Nocturnal frontal lobe epilepsy 1',
'Nephrotic syndrome type 7',
'Sulfocysteinuria',
'Charcotte Marie Tooth Disease 2A',
'Spastic Paraplegia 30',
'Factor X deficiency',
'Hemochromatosis',
'Thyroid dyshormonogenesis',
'Familial erythrocytosis 1',
'Spinocerebellar ataxia',
'Interstitial nephritis karyomegalic',
'HARP syndrome',
'Essential thrombocythemia',
'Primary hyperoxaluria type 1',
'Familial cold autoinflammatory syndrome 3',
'Fabrys Disease',
'Batten disease',
'Splice switching antisense oligonucleotides targeting CLN3']

# Create a DataFrame
mendelian_dx_df = pd.DataFrame(mendelian_dx, columns=['disease'])



In [6]:
# Apply the get_embedding function to each row and create a new column for embeddings
mendelian_dx_df['embeddings'] = mendelian_dx_df['disease'].apply(lambda x: get_embedding(x))


In [7]:
# Save the DataFrame to a .pkl file
mendelian_dx_df.to_pickle('mendelian_diseases_gpt_embeddings.pkl')

In [8]:
import pandas as pd

# Rename the first column in mendelian_dx_df
mendelian_dx_df.rename(columns={"disease": "phenotype"}, inplace=True)

# Concatenate mendelian_dx_df to embeddings_df
combined_df = pd.concat([embeddings_df, mendelian_dx_df], ignore_index=True)



In [9]:
combined_df

,phecode,phenotype,phecode_exclude_range,sex,rollup,leaf,category_number,category,embeddings
0,8.00,Intestinal infection,001-009.99,NaN,1.0,0.0,1.0,infectious diseases,"[0.0068802302703261375, 0.0005461262771859765,..."
1,8.50,Bacterial enteritis,001-009.99,Both,1.0,0.0,1.0,infectious diseases,"[-0.010700925253331661, 1.566567698318977e-05,..."
2,8.51,Intestinal e.coli,001-009.99,Both,1.0,1.0,1.0,infectious diseases,"[0.01900758408010006, -0.0076250554993748665, ..."
3,8.52,Intestinal infection due to C. difficile,001-009.99,Both,1.0,1.0,1.0,infectious diseases,"[0.013732682913541794, -0.013513063080608845, ..."
4,8.60,Viral Enteritis,001-009.99,Both,1.0,1.0,1.0,infectious diseases,"[-0.01282576099038124, -0.0019294519443064928,..."
...,...,...,...,...,...,...,...,...,...
1879,NaN,Primary hyperoxaluria type 1,NaN,NaN,NaN,NaN,NaN,NaN,"[0.019682370126247406, 0.004350563976913691, 0..."
1880,NaN,Familial cold autoinflammatory syndrome 3,NaN,NaN,NaN,NaN,NaN,NaN,"[-0.022045236080884933, 0.012199598364531994, ..."
1881,NaN,Fabrys Disease,NaN,NaN,NaN,NaN,NaN,NaN,"[-0.0041238972917199135, -0.006055511999875307..."
1882,NaN,Batten disease,NaN,NaN,NaN,NaN,NaN,NaN,"[-0.028185836970806122, 0.011104734614491463, ..."


In [10]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Ensure embeddings are in the correct format
def parse_embedding(embedding):
    if isinstance(embedding, str):
        return np.array(eval(embedding))
    return np.array(embedding)

combined_df['embeddings'] = combined_df['embeddings'].apply(parse_embedding)
embeddings = np.vstack(combined_df['embeddings'].values)

# Compute the cosine similarity matrix
cosine_sim_matrix = cosine_similarity(embeddings)

# Create a DataFrame for the similarity matrix with phenotype labels
similarity_df = pd.DataFrame(cosine_sim_matrix, index=combined_df['phenotype'], columns=combined_df['phenotype'])






In [11]:
similarity_df

phenotype,Intestinal infection,Bacterial enteritis,Intestinal e.coli,Intestinal infection due to C. difficile,Viral Enteritis,Intestinal infection due to protozoa,Tuberculosis,Diseases due to other mycobacteria,Leprosy,Septicemia,...,Familial erythrocytosis 1,Spinocerebellar ataxia,Interstitial nephritis karyomegalic,HARP syndrome,Essential thrombocythemia,Primary hyperoxaluria type 1,Familial cold autoinflammatory syndrome 3,Fabrys Disease,Batten disease,Splice switching antisense oligonucleotides targeting CLN3
phenotype,,,,,,,,,,,,,,,,,,,,,
Intestinal infection,1.000000,0.888368,0.907635,0.924155,0.866907,0.930722,0.811887,0.777187,0.789209,0.843248,...,0.755378,0.764904,0.782718,0.732275,0.754622,0.754517,0.774839,0.763328,0.774605,0.726154
Bacterial enteritis,0.888368,1.000000,0.898892,0.860644,0.944254,0.862636,0.813211,0.775117,0.788714,0.838129,...,0.782669,0.778145,0.777446,0.751737,0.768499,0.759658,0.781888,0.763359,0.803710,0.734036
Intestinal e.coli,0.907635,0.898892,1.000000,0.887033,0.868051,0.884783,0.805906,0.777315,0.770433,0.831667,...,0.780562,0.763421,0.786457,0.729654,0.771002,0.762241,0.780507,0.752814,0.765242,0.750408
Intestinal infection due to C. difficile,0.924155,0.860644,0.887033,1.000000,0.835245,0.903699,0.799701,0.795897,0.782623,0.827805,...,0.738767,0.759317,0.779375,0.734184,0.744804,0.750586,0.772701,0.765738,0.761489,0.749480
Viral Enteritis,0.866907,0.944254,0.868051,0.835245,1.000000,0.835917,0.806582,0.750395,0.774189,0.808450,...,0.783609,0.775735,0.780860,0.768104,0.762693,0.757517,0.787048,0.761037,0.780210,0.734658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Primary hyperoxaluria type 1,0.754517,0.759658,0.762241,0.750586,0.757517,0.763306,0.782632,0.759561,0.779784,0.743522,...,0.850711,0.766048,0.797623,0.786213,0.783052,1.000000,0.801983,0.808149,0.777536,0.741353
Familial cold autoinflammatory syndrome 3,0.774839,0.781888,0.780507,0.772701,0.787048,0.758505,0.790832,0.768139,0.777502,0.787947,...,0.849325,0.786846,0.816318,0.807224,0.787851,0.801983,1.000000,0.825635,0.776200,0.782983
Fabrys Disease,0.763328,0.763359,0.752814,0.765738,0.761037,0.761578,0.800506,0.793470,0.804556,0.768395,...,0.833542,0.766788,0.798585,0.785561,0.786949,0.808149,0.825635,1.000000,0.816549,0.719762


In [12]:
# Save the similarity matrix to a CSV file
similarity_df.to_csv('similarity_matrix_with_mendelian_diseases_phecodes_gpt_embeddings.csv')


In [ ]:
# Plotting the heatmap
plt.figure(figsize=(8, 6))
heatmap = sns.heatmap(similarity_df, annot=True, cmap='viridis')
plt.title('Cosine Similarity Matrix Heatmap')
plt.show()

In [ ]:

# def find_similar_phenotypes(df, query, top_n=5):
#     row = df.loc[query]

#     # Sort the values in descending order
#     sorted_row = row.sort_values(ascending=False)

#     # Select the top n values
#     top_values = sorted_row.head(top_n)
#     return top_values

# # Example usage
# query_phenotype = "Intestinal infection"
# top_n = 5
# similar_phenotypes = find_similar_phenotypes(similarity_df, query_phenotype, top_n)
# print(similar_phenotypes)

In [ ]:
import pandas as pd
import difflib

def find_similar_phenotypes(df, query, top_n=5):
    
    print("Search term: " + str(query))
    # Normalize the query to lowercase
    query = query.lower()
    
    # Normalize the dataframe index to lowercase
    normalized_index = df.index.str.lower()
    
    # Find the closest match for the query
    closest_match = difflib.get_close_matches(query, normalized_index, n=1, cutoff=0.6)
    print("Showing results for : " + str(closest_match))
    
    if not closest_match:
        raise ValueError(f"Query phenotype '{query}' not found in the dataframe.")
    
    # Get the actual index of the closest match
    actual_query = df.index[normalized_index == closest_match[0]].tolist()[0]
    
    # Select the row for the query phenotype
    row = df.loc[actual_query]

    # Sort the values in descending order
    sorted_row = row.sort_values(ascending=False)

    # Select the top n values
    top_values = sorted_row.head(top_n)
    
    return top_values

# Example usage
# Assuming similarity_df is your dataframe with similarity scores
query_phenotype = "intestinal infection"
top_n = 10
similar_phenotypes = find_similar_phenotypes(similarity_df, query_phenotype, top_n)
print(similar_phenotypes)
